##### Which model genes have phenotype data but are not associated with a human diseases (and have a human ortholog)

###### Notes
We ingest RGD gene to phenotype but do not get their model_of disease associations, so this should be excluded

Similarly, we don't ingest yeast gene model of disease data as it's not available (at least from sgd) - https://downloads.yeastgenome.org/curation/literature/

In [50]:
# First get all human genes

import requests
from typing import Set, Iterator, Dict
from copy import deepcopy
from prefixcommons import contract_uri, expand_uri
from prefixcommons.curie_util import NoPrefix

# Globals and Constants
SCIGRAPH_URL = 'https://scigraph-data.monarchinitiative.org/scigraph'
SOLR_URL = 'https://solr.monarchinitiative.org/solr/golr/select'

CURIE_MAP = {
    "http://identifiers.org/hgnc/HGNC:": "HGNC"
}

def get_human_genes() -> Set[str]:

    scigraph_service = SCIGRAPH_URL + "/cypher/execute.json"
    query = "MATCH (gene:gene)-[has_taxon:RO:0002162]->(taxon{iri:'http://purl.obolibrary.org/obo/NCBITaxon_9606'}) " \
            "RETURN DISTINCT gene.iri as gene_iri"
    params = {
        "cypherQuery": query,
        "limit": 100000
    }

    request = requests.get(scigraph_service, params=params)
    results = request.json()
    genes = set()
    
    for res in results:
        if res["gene_iri"].startswith("http://flybase.org"):
            continue
        try:
            genes.add(contract_uri(res["gene_iri"], strict=True)[0])
        except NoPrefix:
            for prefix in CURIE_MAP:
                if res["gene_iri"].startswith(prefix):
                    curie = res["gene_iri"].replace(prefix, CURIE_MAP[prefix] + ":")
                    genes.add(curie)
                    break
            
    return genes

human_gene_set = get_human_genes()
# Head list
list(human_gene_set)[0:11]

['HGNC:19826',
 'HGNC:32951',
 'NCBIGene:105372831',
 'NCBIGene:105373954',
 'HGNC:13143',
 'HGNC:32378',
 'HGNC:20036',
 'HGNC:53772',
 'HGNC:18317',
 'HGNC:19827',
 'HGNC:51375']

In [51]:
# Get model genes that are associated to a human disease

model_of_query = {
    "q": "*:*",
    "facet.limit": "5000",
    "facet.field": "subject_gene",
    "fq": [
        "relation:\"RO:0003301\"",  # model_of
        "-subject_gene:HGNC*"  # remove human "models" (cell lines)
    ],
    "facet.mincount": "1",
    "rows": "0",
    "facet": "true",
    "json.nl": "arrarr",
    "wt": "json"
}

solr_request = requests.get(SOLR_URL, params=model_of_query)
response = solr_request.json()
model_genes_w_disease = {facet[0] for facet in response["facet_counts"]["facet_fields"]["subject_gene"]}

len(list(model_genes_w_disease))

2163

In [55]:
# Get all the genes with at least one human ortholog and phenotype association
# (~600k associations)

def get_solr_results(solr, params) -> Iterator[Dict]:
    solr_params = deepcopy(params)  # don't mutate input
    result_count = solr_params['rows']
    if 'start' not in solr_params: solr_params['start'] = 0
    while solr_params['start'] < result_count:
        solr_request = requests.get(solr, params=solr_params)
        response = solr_request.json()
        result_count = response['response']['numFound']
        solr_params['start'] += solr_params['rows']
        if solr_params['start'] % 100000 == 0:
            print("Processed {} docs".format(solr_params['start']))
        for doc in response['response']['docs']:
            yield doc
            
gene_phenotype = {
    "q": "*:*",
    "fl": "subject,subject_taxon_label,subject_ortholog_closure",
    "fq": [
        "subject_category:gene",
        "object_category:phenotype",
        "-subject_taxon:\"NCBITaxon:9606\""  # filter out human g2p
    ],
    "rows": 10000,
    "start": 0,
    "wt": "json"
    }

# need py 3.6
# mod_gene_list: Set[Tuple[str, str]] = []
mod_gene_set = set()

for doc in get_solr_results(SOLR_URL, gene_phenotype):
    if 'subject_ortholog_closure' not in doc:
        continue
    if len(set(doc['subject_ortholog_closure']) & human_gene_set) > 0 \
            and not doc['subject'] in model_genes_w_disease:
        taxon = doc['subject_taxon_label'] if 'subject_taxon_label' in doc else "unknown"
        mod_gene_set.add((doc['subject'], taxon))

len(mod_gene_set)

Processed 100000 docs
Processed 200000 docs
Processed 300000 docs
Processed 400000 docs
Processed 500000 docs
Processed 600000 docs


29449

In [57]:
from collections import Counter 

# unzip and count occurences per species
genes, species = zip(*list(mod_gene_set))

Counter(species)

Counter({'Bos taurus': 993,
         'Caenorhabditis elegans': 4558,
         'Danio rerio': 3390,
         'Drosophila melanogaster': 7951,
         'Equus caballus': 23,
         'Gallus gallus': 128,
         'Mus musculus': 8503,
         'Rattus norvegicus': 563,
         'Saccharomyces cerevisiae S288C': 3013,
         'Sus scrofa': 320,
         'unknown': 7})